<h1>Trabalho de conclusão NLP</h1>

<h2>Turma 10IA</h2>
<h3>Membros:</h3>
<b>Mauricio Gallo - RM: 336168</b></br>
<b>Gustavo Villa - RM: 335645</b></br>
<b>Rafael Favretti - RM: 335332</b></br>

## Leitura e análise dos dados

In [1]:
#Import das bibliotecas
import pandas as pd

#Leitura do dataset
df_review = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/imdb-reviews-pt-br.csv", sep = ",", encoding='utf-8')

#Visualização dos dados
df_review.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [2]:
#Contagem dos dados no dataset
print(df_review.count())

id           49459
text_en      49459
text_pt      49459
sentiment    49459
dtype: int64


In [3]:
#Checagem se há dados nulos no dataset
df_review.isnull().sum()

id           0
text_en      0
text_pt      0
sentiment    0
dtype: int64

In [4]:
#Checagem se há dados NA no dataset
df_review.isna().sum()

id           0
text_en      0
text_pt      0
sentiment    0
dtype: int64

In [5]:
#Verificando as dimensões do dataset
df_review.shape

(49459, 4)

In [6]:
#Contagem inicial das classificações de sentimentos
df_review.sentiment.value_counts()

neg    24765
pos    24694
Name: sentiment, dtype: int64

## Contagem de palavras mais comuns

In [7]:
from collections import Counter
counterPT = Counter()
counterEN = Counter()

#Contagem de palavras mais comuns em português
df_review.text_pt.str.lower().str.split().apply(counterPT.update) 
counterPT.most_common(10)

[('de', 422154),
 ('que', 327261),
 ('e', 311782),
 ('o', 289805),
 ('a', 241726),
 ('um', 225146),
 ('é', 208453),
 ('não', 142163),
 ('em', 140213),
 ('uma', 136215)]

In [8]:
#Contagem de palavras mais comuns em inglês
df_review.text_en.str.lower().str.split().apply(counterEN.update) 
counterEN.most_common(10)

[('the', 612098),
 ('a', 303745),
 ('and', 301853),
 ('of', 272250),
 ('to', 251218),
 ('is', 195962),
 ('in', 171570),
 ('i', 140594),
 ('this', 135274),
 ('it', 125757)]

## Geração de tokens

In [9]:
#Import das bibliotecas para geração dos tokens
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt', quiet=True)

#Criação de uma nova coluna no dataframe para armazenamento dos tokens e aplicação do Word Tokenize para sua geração
df_review['tokens_pt'] = df_review.text_pt.apply(word_tokenize)
df_review['tokens_en'] = df_review.text_en.apply(word_tokenize)

In [10]:
#Tokens PT
df_review['tokens_pt']

0        [Mais, uma, vez, ,, o, Sr., Costner, arrumou, ...
1        [Este, é, um, exemplo, do, motivo, pelo, qual,...
2        [Primeiro, de, tudo, eu, odeio, esses, raps, i...
3        [Nem, mesmo, os, Beatles, puderam, escrever, m...
4        [Filmes, de, fotos, de, latão, não, é, uma, pa...
                               ...                        
49454    [Como, a, média, de, votos, era, muito, baixa,...
49455    [O, enredo, teve, algumas, reviravoltas, infel...
49456    [Estou, espantado, com, a, forma, como, este, ...
49457    [A, Christmas, Together, realmente, veio, ante...
49458    [O, drama, romântico, da, classe, trabalhadora...
Name: tokens_pt, Length: 49459, dtype: object

In [11]:
#Tokens EN
df_review['tokens_en']

0        [Once, again, Mr., Costner, has, dragged, out,...
1        [This, is, an, example, of, why, the, majority...
2        [First, of, all, I, hate, those, moronic, rapp...
3        [Not, even, the, Beatles, could, write, songs,...
4        [Brass, pictures, movies, is, not, a, fitting,...
                               ...                        
49454    [Seeing, as, the, vote, average, was, pretty, ...
49455    [The, plot, had, some, wretched, ,, unbelievab...
49456    [I, am, amazed, at, how, this, movieand, most,...
49457    [A, Christmas, Together, actually, came, befor...
49458    [Working-class, romantic, drama, from, directo...
Name: tokens_en, Length: 49459, dtype: object

## Stopwords

### Stopwords PT

In [12]:
#Download das Stopwords
nltk.download('stopwords', quiet=True)

stopwords_pt = nltk.corpus.stopwords.words('portuguese')

#Exemplo de stopwords
stopwords_pt[:10]

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']

### Stopwords EN

In [13]:
stopwords_en = nltk.corpus.stopwords.words('english')

#Exemplo de stopwords
stopwords_en[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

## Stemização

In [14]:
#Import das bibliotecas
from nltk.stem import PorterStemmer
from nltk.stem.rslp import RSLPStemmer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('rslp', quiet=True)

#Instância Steamer para PT
rslp = RSLPStemmer()

#Instância Steamer para EN
porter = PorterStemmer()

#Função para aplicar steam aos tokens
def stem_pandas_rslp(line):
  return ' '.join([rslp.stem(token) for token in line])

def stem_pandas_porter(line):
  return ' '.join([porter.stem(token) for token in line])

### Stemização PT

In [15]:
df_review['stemmer_pt'] = df_review.tokens_pt.apply(stem_pandas_rslp)

df_review.stemmer_pt.head()

0    mais uma vez , o sr. costn arrum um film por m...
1    est é um exempl do motiv pel qual a maior do f...
2    prim de tud eu odei ess rap imbecil , que não ...
3    nem mesm os beatl pud escrev músic que tod gos...
4    film de fot de lat não é uma palavr apropri pa...
Name: stemmer_pt, dtype: object

### Stemização EN

In [16]:
df_review['stemmer_en'] = df_review.tokens_en.apply(stem_pandas_porter)

df_review.stemmer_en.head()

0    onc again mr. costner ha drag out a movi for f...
1    thi is an exampl of whi the major of action fi...
2    first of all I hate those moron rapper , who c...
3    not even the beatl could write song everyon li...
4    brass pictur movi is not a fit word for them r...
Name: stemmer_en, dtype: object

## Contagem de unigramas com CountVectorizer

### Utilização com Stopwords

#### Texto em PT

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vect_pt_stop = CountVectorizer(ngram_range=(1,1), stop_words=stopwords_pt)
#vect = CountVectorizer(ngram_range=(2,2))
vect_pt_stop.fit(df_review.stemmer_pt)
count_vect_pt_stop = vect_pt_stop.transform(df_review.stemmer_pt)

print('Quantidade unigramas texto PT', count_vect_pt_stop.shape[1])

Quantidade unigramas texto PT 77831


#### Texto em EN

In [32]:
vect_en_stop = CountVectorizer(ngram_range=(1,1), stop_words=stopwords_en)
#vect = CountVectorizer(ngram_range=(2,2))
vect_en_stop.fit(df_review.stemmer_en)
count_vect_en_stop = vect_en_stop.transform(df_review.stemmer_en)

print('Quantidade unigramas texto EN', count_vect_en_stop.shape[1])

Quantidade unigramas texto EN 87576


### Utilização sem Stopwords

#### Texto em PT

In [33]:
vect_pt = CountVectorizer(ngram_range=(1,1))
#vect = CountVectorizer(ngram_range=(2,2))
vect_pt.fit(df_review.stemmer_pt)
count_vect_pt = vect_pt.transform(df_review.stemmer_pt)

print('Quantidade unigramas texto PT', count_vect_pt.shape[1])

Quantidade unigramas texto PT 77937


#### Texto em EN

In [34]:
vect_en = CountVectorizer(ngram_range=(1,1))
#vect = CountVectorizer(ngram_range=(2,2))
vect_en.fit(df_review.stemmer_en)
count_vect_en = vect_en.transform(df_review.stemmer_en)
                                  
print('Quantidade unigramas texto EN', count_vect_en.shape[1])

Quantidade unigramas texto EN 87718


#### Resultado da análise de stopwords com unigramas

Após essa análise verificamos que a utilização ou não de stop words para unigramas não produz uma redução significativa.

## Contagem de bigramas com CountVectorizer

### Utilização com Stopwords

#### Texto em PT

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

vect_pt_stop_bi = CountVectorizer(ngram_range=(2,2), stop_words=stopwords_pt)
vect_pt_stop_bi.fit(df_review.stemmer_pt)
count_vect_pt_stop_bi = vect_pt_stop_bi.transform(df_review.stemmer_pt)

print('Quantidade bigramas texto PT', count_vect_pt_stop_bi.shape[1])

Quantidade bigramas texto PT 2327600


#### Texto em EN

In [36]:
vect_en_stop_bi = CountVectorizer(ngram_range=(2,2), stop_words=stopwords_en)
vect_en_stop_bi.fit(df_review.stemmer_en)
count_vect_en_stop_bi = vect_en_stop_bi.transform(df_review.stemmer_en)

print('Quantidade bigramas texto EN', count_vect_en_stop_bi.shape[1])

Quantidade bigramas texto EN 2548514


### Utilização sem Stopwords

#### Texto em PT

In [37]:
vect_pt_bi = CountVectorizer(ngram_range=(2,2))
vect_pt_bi.fit(df_review.stemmer_pt)
count_vect_pt_bi = vect_pt_bi.transform(df_review.stemmer_pt)

print('Quantidade bigramas texto PT', count_vect_pt_bi.shape[1])

Quantidade bigramas texto PT 1855113


#### Texto em EN

In [38]:
vect_en_bi = CountVectorizer(ngram_range=(2,2))
vect_en_bi.fit(df_review.stemmer_en)
count_vect_en_bi = vect_en_bi.transform(df_review.stemmer_en)
                                  
print('Quantidade bigramas texto EN', count_vect_en_bi.shape[1])

Quantidade bigramas texto EN 2005774


#### Resultado da análise de stopwords com bigramas

Após essa análise verificamos que a utilização de stop words para bigramas produz uma redução significativa.

# Treino modelos

In [24]:
#Import da biblioteca
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

## Unigramas

## CountVectorizer

### PT

In [25]:
#Cópia do dataset
df_uni_pt = df_review

#Separação entre teste e treino
X_train_uni_pt, X_test_uni_pt, y_train_uni_pt, y_test_uni_pt = train_test_split(
count_vect_pt_stop,
df_uni_pt["sentiment"],
test_size = 0.2,
random_state = 42)

X_train_uni_pt

<39567x77831 sparse matrix of type '<class 'numpy.int64'>'
	with 4055771 stored elements in Compressed Sparse Row format>

In [42]:
#Treino da árvore de decisão
tree_uni_pt = DecisionTreeClassifier(random_state = 42)
tree_uni_pt.fit(X_train_uni_pt, y_train_uni_pt)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [43]:
#Avaliação da acurácia do modelo
tree_uni_pt.score(X_test_uni_pt, y_test_uni_pt)

0.7164375252729478

### EN

In [44]:
#Cópia do dataset
df_uni_en = df_review

#Separação entre teste e treino
X_train_uni_en, X_test_uni_en, y_train_uni_en, y_test_uni_en = train_test_split(
count_vect_en_stop,
df_uni_en["sentiment"],
test_size = 0.2,
random_state = 42)

X_train_uni_en

<39567x87576 sparse matrix of type '<class 'numpy.int64'>'
	with 3851756 stored elements in Compressed Sparse Row format>

In [45]:
#Treino da árvore de decisão
tree_uni_en = DecisionTreeClassifier(random_state = 42)
tree_uni_en.fit(X_train_uni_en, y_train_uni_en)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [46]:
#Avaliação da acurácia do modelo
tree_uni_en.score(X_test_uni_en, y_test_uni_en)

0.7308936514355034

## Bigramas

### PT

In [47]:
#Cópia do dataset
df_bi_pt = df_review

#Separação entre teste e treino
X_train_bi_pt, X_test_bi_pt, y_train_bi_pt, y_test_bi_pt = train_test_split(
count_vect_pt_stop_bi,
df_bi_pt["sentiment"],
test_size = 0.2,
random_state = 42)

X_train_bi_pt

<39567x2975485 sparse matrix of type '<class 'numpy.int64'>'
	with 4885457 stored elements in Compressed Sparse Row format>

In [48]:
#Treino da árvore de decisão
tree_bi_pt = DecisionTreeClassifier(random_state = 42)
tree_bi_pt.fit(X_train_bi_pt, y_train_bi_pt)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [50]:
#Avaliação da acurácia do modelo
tree_bi_pt.score(X_test_bi_pt, y_test_bi_pt)

0.7354427820460978

### EN

In [51]:
#Cópia do dataset
df_bi_en = df_review

#Separação entre teste e treino
X_train_bi_en, X_test_bi_en, y_train_bi_en, y_test_bi_en = train_test_split(
count_vect_en_stop_bi,
df_bi_en["sentiment"],
test_size = 0.2,
random_state = 42)

X_train_bi_en

<39567x2548514 sparse matrix of type '<class 'numpy.int64'>'
	with 4812369 stored elements in Compressed Sparse Row format>

In [52]:
#Treino da árvore de decisão
tree_bi_en = DecisionTreeClassifier(random_state = 42)
tree_bi_en.fit(X_train_bi_en, y_train_bi_en)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [53]:
#Avaliação da acurácia do modelo
tree_bi_en.score(X_test_bi_en, y_test_bi_en)

0.7382733522038011

# Análise com stopwords Spacy

In [59]:
#import da biblioteca Spacy
import spacy

## PT

In [71]:
#Quantidades de stopwords
print(len(stopwords_pt))

#Pegando stopwords do spacy
stopwords_space_pt = spacy.load('pt_core_news_sm').Defaults.stop_words

#Lista stopwords concatenada
stopwords_full_pt = set(stopwords_space_pt).union(set(stopwords_pt))

#Count de stopwords atual
print(len(stopwords_full_pt))

204
499


Após isso resolvemos executar novamente o modelo com bigramas pois foi o mais efetivo até o momento

In [78]:
from sklearn.feature_extraction.text import CountVectorizer

vect_pt_stop_bi = CountVectorizer(ngram_range=(2,2), stop_words=stopwords_full_pt)
vect_pt_stop_bi.fit(df_review.stemmer_pt)
count_vect_pt_stop_bi = vect_pt_stop_bi.transform(df_review.stemmer_pt)

print('Quantidade bigramas texto PT', count_vect_pt_stop_bi.shape[1])

Quantidade bigramas texto PT 2280339


In [79]:
#Cópia do dataset
df_bi_pt = df_review

#Separação entre teste e treino
X_train_bi_pt, X_test_bi_pt, y_train_bi_pt, y_test_bi_pt = train_test_split(
count_vect_pt_stop_bi,
df_bi_pt["sentiment"],
test_size = 0.2,
random_state = 42)

X_train_bi_pt

<39567x2280339 sparse matrix of type '<class 'numpy.int64'>'
	with 4836559 stored elements in Compressed Sparse Row format>

In [80]:
#Treino da árvore de decisão
tree_bi_pt = DecisionTreeClassifier(random_state = 42)
tree_bi_pt.fit(X_train_bi_pt, y_train_bi_pt)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [81]:
#Avaliação da acurácia do modelo
tree_bi_pt.score(X_test_bi_pt, y_test_bi_pt)

0.7197735543873838

## EN

In [82]:
#Quantidades de stopwords
print(len(stopwords_en))

#Pegando stopwords do spacy
stopwords_space_en = spacy.load('en_core_web_md').Defaults.stop_words

#Lista stopwords concatenada
stopwords_full_en = set(stopwords_space_en).union(set(stopwords_en))

#Count de stopwords atual
print(len(stopwords_full_en))

179
382


In [83]:
vect_en_stop_bi = CountVectorizer(ngram_range=(2,2), stop_words=stopwords_full_en)
vect_en_stop_bi.fit(df_review.stemmer_en)
count_vect_en_stop_bi = vect_en_stop_bi.transform(df_review.stemmer_en)

print('Quantidade bigramas texto EN', count_vect_en_stop_bi.shape[1])

Quantidade bigramas texto EN 2463386


In [84]:
#Cópia do dataset
df_bi_en = df_review

#Separação entre teste e treino
X_train_bi_en, X_test_bi_en, y_train_bi_en, y_test_bi_en = train_test_split(
count_vect_en_stop_bi,
df_bi_en["sentiment"],
test_size = 0.2,
random_state = 42)

X_train_bi_en

<39567x2463386 sparse matrix of type '<class 'numpy.int64'>'
	with 4306741 stored elements in Compressed Sparse Row format>

In [85]:
#Treino da árvore de decisão
tree_bi_en = DecisionTreeClassifier(random_state = 42)
tree_bi_en.fit(X_train_bi_en, y_train_bi_en)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [86]:
#Avaliação da acurácia do modelo
tree_bi_en.score(X_test_bi_en, y_test_bi_en)

0.7287707238172261

# Spacy

In [91]:
spacy_pt = spacy.load('pt_core_news_sm')
spacy_en = spacy.load('en_core_web_md')

#Função para realizar a lematização em pt
def lemmatizer_text_pt(text):        
  sent = []
  doc = spacy_pt(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)


#Função para realizar a lematização em en
def lemmatizer_text_en(text):        
  sent = []
  doc = spacy_en(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

df_review['text_lemma_pt'] = df_review.text_pt.apply(lemmatizer_text_pt)
df_review['text_lemma_en'] = df_review.text_en.apply(lemmatizer_text_en)

In [93]:
#Checando o head da co
print(df_review['text_lemma_pt'].head())
print(df_review['text_lemma_en'].head())

0    Mais umar vez , o Sr. Costner arrumar um filma...
1    Este ser um exemplo do motivar pelar qual o ma...
2    Primeiro de tudo eu odiar esse rap imbecil , q...
3    Nem mesmo o Beatles poder escrever músico que ...
4    Filmes de foto de latão não ser umar palavra a...
Name: text_lemma_pt, dtype: object
0    once again Mr. Costner have drag out a movie f...
1    this be an example of why the majority of acti...
2    first of all -PRON- hate those moronic rapper ...
3    not even the Beatles could write song everyone...
4    brass picture movie be not a fitting word for ...
Name: text_lemma_en, dtype: object


In [96]:
#Import
from sklearn.metrics import accuracy_score

#Copia do datasource
df_spacy_lemma = df_review

vect_spacy_lemma = CountVectorizer(ngram_range=(1,2), stop_words=stopwords_full_pt) # exemplo 2.1: vetorização e combinação de unigrama e bigrama sem stopwords
vect_spacy_lemma.fit(df_spacy_lemma.text_lemma_pt)
text_vect_spacy_lemma = vect_spacy_lemma.transform(df_spacy_lemma.text_lemma_pt)

#Separação dos dados
X_train_spacy_lemma,X_test_spacy_lemma,y_train_spacy_lemma,y_test_spacy_lemma = train_test_split(
    text_vect_spacy_lemma, 
    df_spacy_lemma["sentiment"], 
    test_size = 0.2,
    random_state = 42
  )

In [97]:
#Treino do modelo
spacy_tree = DecisionTreeClassifier(random_state=42)
spacy_tree.fit(X_train_spacy_lemma, y_train_spacy_lemma)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [99]:
#Teste acurácia
y_prediction = spacy_tree.predict(X_test_spacy_lemma)

accuracy = accuracy_score(y_prediction, y_test_spacy_lemma)

print(accuracy)

0.7259401536595228


# Teste sem stopwords

In [100]:
#Copia do datasource
df_spacy_lemma = df_review

vect_spacy_lemma = CountVectorizer(ngram_range=(1,2))
vect_spacy_lemma.fit(df_spacy_lemma.text_lemma_pt)
text_vect_spacy_lemma = vect_spacy_lemma.transform(df_spacy_lemma.text_lemma_pt)

#Separação dos dados
X_train_spacy_lemma,X_test_spacy_lemma,y_train_spacy_lemma,y_test_spacy_lemma = train_test_split(
    text_vect_spacy_lemma, 
    df_spacy_lemma["sentiment"], 
    test_size = 0.2,
    random_state = 42
  )

In [101]:
#Treino do modelo
spacy_tree = DecisionTreeClassifier(random_state=42)
spacy_tree.fit(X_train_spacy_lemma, y_train_spacy_lemma)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [102]:
#Teste acurácia
y_prediction = spacy_tree.predict(X_test_spacy_lemma)

accuracy = accuracy_score(y_prediction, y_test_spacy_lemma)

print(accuracy)

0.727456530529721


# Conclusão

Concluímos que o melhor cenário a ser utilizado é execução de tokenização com bigramas, resultando em uma acurácia de 73% tanto para português quanto para inglês.